# $\textit{Predicción a nivel de palabra}$

In [ ]:
import zipfile
import wget
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
filename = wget.download(url=url)
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(path="./")

In [ ]:
with open("Sherlock.txt", 'r', encoding='utf-8') as file:
    book = file.read()

In [ ]:
token = Tokenizer(num_words=30000,
                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
                  split=' ', char_level=False, oov_token="UNK",
                  document_count=0)

token.fit_on_texts([book])

In [ ]:
sequences = np.array(token.texts_to_sequences([book])[0])

In [ ]:
def create_dataset(text, window=100):
  inputs = []
  outputs = []
  
  for i in range(len(text)-window):
    inputs.append(text[i:i+window])
    outputs.append(text[i+1:i+window+1])
  return list(inputs), list(outputs)

In [ ]:
window_size = 100
X, y = create_dataset(sequences, window_size) #Creamos una secuencia de largo window, con el texto de referencia en X y el texto mas 1 dato mas en y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, shuffle=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense, Input, Concatenate, Dot, RepeatVector, TimeDistributed, Multiply, Lambda, Flatten, Activation, Reshape, BatchNormalization
import tensorflow.keras.backend as K
from tensorflow.keras.activations import softmax
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model

def get_model(dropout_p = 0.1, output = len(chars), input_shape = (100, 71), RNN_units=200):
    model = Sequential()
    model.add(SimpleRNN(RNN_units, input_shape=input_shape, return_sequences=True, name='RNN',
                        dropout=dropout_p, recurrent_dropout=dropout_p ))
    model.add(Dense(output, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model